In [1]:
import pg_toolkit as pgt
pgt.toolkit_config.set_pg_conn_string("dbname='tenders'")

In [2]:
import json
import glob

data = [json.load(open(x)) for x in glob.glob('/Users/tilarids/data/tenders/*/*.json')]

In [3]:
pgt.import_json_into_pg('tenders', data, lambda record: record['tenderID'], create_table=True, skip_duplicates=True)

In [4]:
pgt.pg_query("""
    SELECT 
        t.id,
        t.data->'tenderID' as tenderID,
        t.data->'value'->'amount' as amount,
        t.data->'status' as status,
        tenderer->'name' as tenderer_name,
        bid->'value'->'amount' as bid_amount,
        tenderer->'identifier'->'id' as tenderer_id,
        award->'value'->'amount' as award_amount
    FROM tenders t,
        jsonb_array_elements(t.data->'bids') bid,
        jsonb_array_elements(bid->'tenderers') tenderer,
        jsonb_array_elements(t.data->'awards') award
    LIMIT 10""")

,id,tenderid,amount,status,tenderer_name,bid_amount,tenderer_id,award_amount
0,UA-2015-08-31-000073,UA-2015-08-31-000073,46305.00,cancelled,АВ Металл групп,38475.00,36441934,38475.00
1,UA-2015-10-20-000360,UA-2015-10-20-000360,213854.00,active.qualification,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""ВЛАДА...",212000.40,39290839,212000.40
2,UA-2015-10-16-000054,UA-2015-10-16-000054,2871.00,unsuccessful,"ФОП ""Малиновський Олександр Леонідович""",2871.00,3256706236,2871.00
3,UA-2015-10-22-000040,UA-2015-10-22-000040,3080.00,active.qualification,"ПАТ ""МЕДИЦИНА""",2206.13,23538297,2206.13
4,UA-2015-10-22-000040,UA-2015-10-22-000040,3080.00,active.qualification,"Товариство з обмеженою відповідальністю ""АКЦЕП...",2580.00,38506076,2206.13
5,UA-2015-10-22-000040,UA-2015-10-22-000040,3080.00,active.qualification,ФОП Журавель Євген Валерійович,2800.00,3189918412,2206.13
6,UA-2015-10-16-000163,UA-2015-10-16-000163,1598930.04,unsuccessful,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «МД ІС...,1590935.39,0032040840,1350000.00
7,UA-2015-10-16-000163,UA-2015-10-16-000163,1598930.04,unsuccessful,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «МД ІС...,1590935.39,0032040840,1581505.43
8,UA-2015-10-16-000163,UA-2015-10-16-000163,1598930.04,unsuccessful,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «МД ІС...,1590935.39,0032040840,1588000.00
9,UA-2015-10-16-000163,UA-2015-10-16-000163,1598930.04,unsuccessful,ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «МД ІС...,1590935.39,0032040840,1590935.39


In [5]:
z = pgt.pg_query_by_id('tenders', 'UA-2015-10-20-000200')

In [6]:
z['value']['amount']=10505.42
pgt.pg_update_record('tenders', 'UA-2015-10-20-000200', z)

In [10]:
pgt.pg_query_by_id('tenders', 'UA-2015-10-20-000200')['value']['amount']

10505.42